# 构造微调训练数据集

借助 ChatGPT 和 GPT API 我们可以实现自动化批量构造训练数据集。

下面我们以中国古典哲学数据集为例，展示了自动构造训练集的主要流程：

- 使用 LangChain 构造训练数据样例
    - 基于 ChatGPT 设计 `System Role` 提示词
    - 使用 `OpenAI GPT-4o-mini` 生成基础数据
    - 解析 OpenAI GPT 生成的训练数据
    - 持久化存储`dataset.csv`训练数据集文件
    - 使用 ChatGPT 实现训练数据多样化
- 自动化批量生成训练数据集
    - 整理收集原始数据`raw_data.txt`
    - 自动解析原始数据样例 `raw_data_content[]`
    - 设计 `gen_data` 训练数据生成器函数
    - 设计训练数据生成流水线

最佳实践参考：

- 使用 GPT-4o-mini 生成基础数据：https://platform.openai.com/playground/p/2c7XNPgo6Y2iDxILiWfD3iPu?model=gpt-4o-mini&mode=chat
- 使用 ChatGPT 生成数据处理代码和相关文本整理：https://chat.openai.com/share/cdfd2d1d-a75e-4cee-be49-539c010ca1b1
- GPT API 价格: https://openai.com/pricing

## 使用 OpenAI SDk 构造训练数据

In [1]:
from openai import OpenAI

api_key = "sk-19xxxxxxxxxxxxxxe8"
client = OpenAI(base_url="https://api.deepseek.com/v1", api_key=api_key)

In [2]:
response = client.chat.completions.create(
  model="deepseek-chat",
  messages=[
    {
      "role": "system",
      "content": "你是中国古典哲学大师，尤其擅长周易的哲学解读。\n\n接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。\n\n示例输入：\n\n师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。\n师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。\n\n期待结果：\n\ncontent:\"师卦\"\nsummary:\"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。\n\n师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。\n\n师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。\""
    },
    {
      "role": "user",
      "content": "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"
    },
    {
      "role": "assistant",
      "content": "content:\"蒙卦\"\nsummary:\"在周易中，蒙卦象征着教育启蒙和智慧的初期阶段，由两个异卦相叠组成：下卦坎（水）和上卦艮（山）。坎象征水，艮象征山，代表了山下有水源，水流出山。这预示着在事物发展的起初阶段，人们会陷入蒙昧状态，因此教育成为当务之急。只有培养纯正无邪的品质，才能走上通达之路，如同泉水汇成江河。\n\n蒙卦的核心哲学是：事物发展的初期阶段必然处于蒙昧状态，但如果能把握时机，行动切合时宜，就能启蒙通达。蒙卦紧随始生卦《屯》之后，《序卦》中解释道：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”这说明物在幼稚阶段都会经历蒙昧状态，找它蒙即是稚嫩。\n\n《象》中解释说：山下出泉，蒙；君子以果行育德，意味着像泉水从山下流出一样，启蒙教育有助于培育美德。\""
    },
    {
      "role": "user",
      "content": "需卦，本卦是异卦相叠，上卦为坎，下卦为乾。乾卦象征天空辽阔，坎卦象征乌云密布。雨水之于传统农业社会而言，是最大的需求，因此用这个意象来表现。古人引申为雨将下，万物皆待，所以叫需。\n需，古体字结构是“雨上天下”，从雨而声。如苍天下雨，滋养万物及人。从人的角度看，天下雨，则不易出行耕作，因此在家喝酒吃肉、静待天时。所以，这一卦的基本象征就是等待、不进，还有饮食之义。\n \n需卦位于蒙卦之后，《序卦》之中这样解释道：“物稚不可不养也，故受之以需。需者，饮食之道也。”有所需要，也是有所等待。\n《象》曰：云上于天，需;君子以饮食宴乐。\n《象》中这段话的意思是说：需卦的卦象是乾(天)下坎(水)上，为水在天上之表象。水汽聚集天上成为云层，密云満天，但还没有下雨，需要等待;君子在这个时候需要吃喝，饮酒作乐，即在等待的时候积蓄力量。\n需卦象征等待，启示人们守正待机的道理，属于中上卦。《象》这样评断此卦：明珠土埋日久深，无光无亮到如今，忽然大风吹土去，自然显露有重新。\n"
    },
    {
      "role": "assistant",
      "content": "content:\"需卦\"\nsummary:\"在周易中，需卦是一个富有深意的卦象，由两个异卦相叠组成：下卦乾（天）和上卦坎（水）。乾象征广阔的天空，而坎象征乌云密布，暗示着迫切需要雨水的意象。古人将雨水视为万物生长的必需，因此命名为“需”。需卦的基本象征是等待和饮食之道。\n\n需卦位于蒙卦之后，《序卦》中解释为“物稚不可不养也，故受之以需。需者，饮食之道也。”说明物在幼稚状态需要得到滋养，因此受到需卦的启示。《象》中描述：云上于天，需；君子以饮食宴乐。意味着天空乌云密布，需要等待雨水，而君子则在等待的过程中积蓄力量，享受饮食和欢乐。\n\n需卦的核心哲学是：等待，启示着守正待机的道理。它属于中上卦，象征着埋藏已久的明珠，经过大风吹去尘埃后，重新显露光芒。\""
    }
  ],
  temperature=1,
  max_tokens=4095,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

---

## 使用 LangChain 构造训练数据

In [4]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(model="deepseek-chat",
                  base_url="https://api.deepseek.com/v1", api_key=api_key,
                  temperature=1,
                  max_tokens=4095)

In [6]:
system_content = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：

师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：

content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""


In [7]:
# 原始数据
raw_content = "蒙卦是教育启蒙的智慧，艮为山，坎为泉，山下出泉。泉水始流出山，则必将渐汇成江河,正如蒙稚渐启，又山下有险，因为有险停止不前，所以蒙昧不明。事物发展的初期阶段，必然蒙昧，所以教育是当务之急，养学生纯正无邪的品质，是治蒙之道。\n蒙卦，这个卦是异卦相叠，下卦为坎，上卦为艮。艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜;因此，具有启蒙和通达的卦象。\n《蒙》卦是《屯》卦这个始生卦之后的第二卦。《序卦》中说：“物生必蒙，故受之以蒙。蒙者，蒙也，特之稚也。”物之幼稚阶段，有如蒙昧未开的状态，在人则是指童蒙。\n《象》中这样解释蒙卦：山下出泉，蒙；君子以果行育德。"

In [8]:
messages = [
    SystemMessage(
        content=system_content
    ),
    HumanMessage(
        content=raw_content
    ),
]

In [9]:
ai_message = chat.invoke(messages)  # 使用 invoke 替代直接调用

### 解析 OpenAI GPT 生成的训练数据

In [10]:
ai_message.content

'content:"蒙卦"\nsummary:"蒙卦在周易中象征着启蒙与教育的智慧，其卦象由下卦坎（水）和上卦艮（山）组成。这一卦象生动地描绘了\'山下出泉\'的景象：泉水从山间初流，终将汇成江河，正如人的蒙稚状态需要逐步开启。坎卦代表险阻，艮卦代表静止，暗示在事物发展的初期阶段，人们往往会因面临险阻而停滞不前，处于蒙昧不明的状态。\n\n蒙卦的核心哲学在于：万物初生必然经历蒙昧时期，因此教育成为至关重要的任务。培养纯正无邪的品质被视为\'治蒙之道\'。虽然卦象显示山下有险，但关键在于把握时机，采取符合时宜的行动，这样才能实现启蒙通达的效果。\n\n在卦序中，蒙卦紧随屯卦（始生卦）之后，《序卦》解释道：\'物生必蒙，故受之以蒙。\'这揭示了事物发展从诞生到启蒙的自然过程。而《象传》\'山下出泉，蒙；君子以果行育德\'的解读，则进一步强调了君子应当通过果断的行动来培育德行，完成启蒙的使命。"'

In [11]:
text = ai_message.content

# 分割字符串来找到content和summary的位置
content_start = text.find('content:"') + len('content:"')
content_end = text.find('"\nsummary:')
summary_start = text.find('summary:"') + len('summary:"')
summary_end = text.rfind('"')

# 提取并存储content和summary
content = text[content_start:content_end].strip()
summary = text[summary_start:summary_end].strip()

print("Content:", content)
print("Summary:", summary)


Content: 蒙卦
Summary: 蒙卦在周易中象征着启蒙与教育的智慧，其卦象由下卦坎（水）和上卦艮（山）组成。这一卦象生动地描绘了'山下出泉'的景象：泉水从山间初流，终将汇成江河，正如人的蒙稚状态需要逐步开启。坎卦代表险阻，艮卦代表静止，暗示在事物发展的初期阶段，人们往往会因面临险阻而停滞不前，处于蒙昧不明的状态。

蒙卦的核心哲学在于：万物初生必然经历蒙昧时期，因此教育成为至关重要的任务。培养纯正无邪的品质被视为'治蒙之道'。虽然卦象显示山下有险，但关键在于把握时机，采取符合时宜的行动，这样才能实现启蒙通达的效果。

在卦序中，蒙卦紧随屯卦（始生卦）之后，《序卦》解释道：'物生必蒙，故受之以蒙。'这揭示了事物发展从诞生到启蒙的自然过程。而《象传》'山下出泉，蒙；君子以果行育德'的解读，则进一步强调了君子应当通过果断的行动来培育德行，完成启蒙的使命。


### 持久化存储训练数据集文件

In [12]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

# 新建CSV文件并写入数据
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入标题行
    writer.writerow(['content', 'summary'])
    # 写入数据行
    writer.writerow([content, summary])


### 数据增强：构造多样化的提问方式

In [13]:
def generate_question_summary_pairs(content, summary):
    """
    生成20对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含20对提问和总结的列表。
    """
    # 20种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "{}和教育启蒙有什么联系？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}在周易中怎样表达教育的概念？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "{}和启蒙教育之间有何联系？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。",
        "{}在周易哲学中扮演什么角色？"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]

    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions]

    return question_summary_pairs

In [14]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
pairs = generate_question_summary_pairs(content, summary)

# 将结果写入CSV文件
with open('test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['content', 'summary'])
    for pair in pairs:
        writer.writerow(pair)


## 自动化批量生成训练数据流水线

原始数据来源：https://www.zhouyi.cc/zhouyi/yijing64/4103.html

In [15]:
# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)

In [16]:
# 输出结果以验证
for i, sample in enumerate(raw_content_data[:5]):  # 打印前5个样例以检查
    print(f"样例 {i+1}:")
    print(sample)
    print("------")


样例 1:
蒙卦原文
蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。
象曰：山下出泉，蒙。君子以果行育德。
白话文解释
蒙卦：通泰。不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。第一次占筮，神灵告诉了他。轻慢不敬的再三占筮，轻慢不敬的占筮，神灵就不会告诉他。但还是吉利的卜问。
《象辞》说：上卦为艮，象征山；下卦为坎，象征泉。山下有泉，泉水喷涌而出，这是蒙卦的卦象。君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。
《断易天机》解
蒙卦艮上坎下，为离宫四世卦。蒙即蒙昧，主回还往复，疑惑不前，多忧愁过失，乃是凶卦。
北宋易学家邵雍解
智慧未开，蒙昧闭塞；犹豫不决，缺乏果断。
得此卦者，智慧犹如童蒙，不辨是非，迷失方向；若能顺贤师良友之教，启其聪明则亨通。
台湾国学大儒傅佩荣解
时运：蓄积德行，出而用世。
财运：矿山生意，果决则吉。
家宅：君子居吉；婚姻之始。
身体：驱去邪热，可保平安。
传统解卦
这个卦是异卦（下坎上艮）相叠，艮是山的形象，喻止；坎是水的形象，喻险。卦形为山下有险，仍不停止前进，是为蒙昧，故称蒙卦。但因把握时机，行动切合时宜，因此，具有启蒙和通达的卦象。
大象：蒙者，昏而无所见也，故宜「启蒙」。
运势：初时迷惑不知方向，须忍耐待机而动，凡事多听取别人意见，则运可通。
事业：事业开始，混乱无序，危机四伏，以勇敢坚毅的行动可以扭转局面。然而必须接受严格教育，培养这种奋发图强的精神。务必脚踏实地，最忌好高骛远，否则会陷入孤立无援的境地。
经商：务必小心谨慎，不得急功近利，尤其应树立高尚的商业道德，以良好的信誉提高竞争力而取胜。
求名：必须接受良好的基础教育，陶冶情操。且动机纯正，可以达到目的。
婚恋：注意考察对方品德，不可以金钱为诱铒。夫妻需相互宽容、理解。
决策：有时会陷入迷惘困顿的境地，加上胆小、不果断，往往误事。如能接受长辈的教诲，甚至严酷的考验，抛弃疑惧的心理，等待适当时机，必然一帆风顺。
------
样例 2:
屯卦原文
屯。元，亨，利，贞。勿用，有攸往，利建侯。
象曰：云，雷，屯；君子以经纶。
白话文解释
屯卦。大吉大利，吉利的占卜。不利于出门。有利于建国封侯。
《象辞》说：屯的上卦为坎，坎为云，下卦为震，震为雷。云行于上，雷动于下，是屯卦的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国

### 将以上的所有模块，整合到一起，自动化生成数据

In [17]:

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# 初始化LangChain的GPT-4o-mini调用
chat = ChatOpenAI(model="deepseek-chat",
                  base_url="https://api.deepseek.com/v1", api_key=api_key,
                  temperature=1,
                  max_tokens=4095)

def gen_data(raw_content):
    """
    使用LangChain GPT-4o-mini调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-4o-mini模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [18]:
# 示例调用（使用 raw_data.txt 中解析的数据样例）
generated_content = gen_data(raw_content_data[0])
print(generated_content)

/tmp/ipykernel_347662/274895050.py:52: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = chat(messages)


```json
{
  "content": "蒙卦",
  "summary": "在周易中，蒙卦是一个象征启蒙与教育的卦象，由下卦坎（水）和上卦艮（山）组成。坎代表险难，艮代表停止，卦象为山下有险，意味着在蒙昧不明的情况下需要谨慎行事。蒙卦的核心哲学是：启蒙不是主动去教导愚昧之人，而是愚昧之人主动求教。第一次求教会有回应，但若轻慢不敬地再三求问，则不会得到回应。尽管如此，此卦仍预示着吉利的结果。\n\n《象辞》解释蒙卦为“山下出泉”，象征君子应当效仿山泉果敢前行的精神，以坚定的行动培养品德。《断易天机》认为蒙卦主疑惑不前，多忧愁过失，是凶卦。邵雍则指出得此卦者智慧未开，需贤师良友的引导才能通达。傅佩荣从时运、财运、家宅、身体等方面解析，强调蓄积德行、果决行动的重要性。\n\n传统解卦认为蒙卦虽象征蒙昧，但把握时机、行动切合时宜则能启蒙通达。运势方面，初时可能迷惑，需忍耐待机，多听取他人意见。事业上需勇敢坚毅，接受严格教育，脚踏实地。经商要谨慎，树立高尚道德。求名需纯正动机和良好教育。婚恋需注重品德考察，夫妻相互理解。决策时可能陷入迷惘，需接受长辈教诲，抛弃疑惧，等待时机。"
}
```


In [19]:
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content，提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 分割字符串来找到content和summary的位置
    content_start = ai_message_content.find('content:"') + len('content:"')
    content_end = ai_message_content.find('"\nsummary:')
    summary_start = ai_message_content.find('summary:"') + len('summary:"')
    summary_end = ai_message_content.rfind('"')

    # 提取并存储content和summary
    content = ai_message_content[content_start:content_end].strip()
    summary = ai_message_content[summary_start:summary_end].strip()

    return content, summary


In [20]:
# 示例调用（使用假设的gen_data函数返回的文本）
content, summary = dataset_parser(generated_content)
print("Content:", content)
print("Summary:", summary)

Content: {
  "content": "蒙卦",
  "summary": "在周易中，蒙卦是一个象征启蒙与教育的卦象，由下卦坎（水）和上卦艮（山）组成。坎代表险难，艮代表停止，卦象为山下有险，意味着在蒙昧不明的情况下需要谨慎行事。蒙卦的核心哲学是：启蒙不是主动去教导愚昧之人，而是愚昧之人主动求教。第一次求教会有回应，但若轻慢不敬地再三求问，则不会得到回应。尽管如此，此卦仍预示着吉利的结果。\n\n《象辞》解释蒙卦为“山下出泉”，象征君子应当效仿山泉果敢前行的精神，以坚定的行动培养品德。《断易天机》认为蒙卦主疑惑不前，多忧愁过失，是凶卦。邵雍则指出得此卦者智慧未开，需贤师良友的引导才能通达。傅佩荣从时运、财运、家宅、身体等方面解析，强调蓄积德行、果决行动的重要性。\n\n传统解卦认为蒙卦虽象征蒙昧，但把握时机、行动切合时宜则能启蒙通达。运势方面，初时可能迷惑，需忍耐待机，多听取他人意见。事业上需勇敢坚毅，接受严格教育，脚踏实地。经商要谨慎，树立高尚道德。求名需纯正动机和良好教育。婚恋需注重品德考察，夫妻相互理解。决策时可能陷入迷惘，需接受长辈教诲，抛弃疑惧，等待时机。"
}
``
Summary: {
  "content": "蒙卦",
  "summary": "在周易中，蒙卦是一个象征启蒙与教育的卦象，由下卦坎（水）和上卦艮（山）组成。坎代表险难，艮代表停止，卦象为山下有险，意味着在蒙昧不明的情况下需要谨慎行事。蒙卦的核心哲学是：启蒙不是主动去教导愚昧之人，而是愚昧之人主动求教。第一次求教会有回应，但若轻慢不敬地再三求问，则不会得到回应。尽管如此，此卦仍预示着吉利的结果。\n\n《象辞》解释蒙卦为“山下出泉”，象征君子应当效仿山泉果敢前行的精神，以坚定的行动培养品德。《断易天机》认为蒙卦主疑惑不前，多忧愁过失，是凶卦。邵雍则指出得此卦者智慧未开，需贤师良友的引导才能通达。傅佩荣从时运、财运、家宅、身体等方面解析，强调蓄积德行、果决行动的重要性。\n\n传统解卦认为蒙卦虽象征蒙昧，但把握时机、行动切合时宜则能启蒙通达。运势方面，初时可能迷惑，需忍耐待机，多听取他人意见。事业上需勇敢坚毅，接受严格教育，脚踏实地。经商要谨慎，树立高尚道德。求名需纯正动机和良好教育。婚恋需注重品德考察，夫妻相互理解。决策时可能陷入迷惘，需接受长辈教诲，抛弃疑惧，等待时机。


In [21]:
import csv
import datetime
import os

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/zhouyi_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = dataset_parser(ai_message_content)
            
            print("Content:", content)
            print("Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)


In [22]:
# 执行主函数
main()

Content: 蒙卦
Summary: 在周易中，蒙卦是一个充满象征意义的卦象，由下卦坎（水）和上卦艮（山）组成。这一卦象代表着“蒙”，即蒙昧和启蒙的状态。坎卦象征着险难，而艮卦象征着停止，暗示着在险难面前需要停下来进行启蒙和思考。

蒙卦的核心哲学是：启蒙不是由智者主动施与，而是由蒙昧者主动寻求。第一次真诚的求问会得到回应，但反复轻慢的求问则不会得到答案。这一卦象强调了真诚和尊重在学习和启蒙过程中的重要性。

从象征意义上看，蒙卦如同山下涌出的泉水，君子应当效仿这种果敢前行的精神，以培养自己的品德。在《断易天机》中，蒙卦被视为凶卦，主疑惑不前，多忧愁过失。然而，北宋易学家邵雍认为，若能接受贤师良友的教导，开启智慧，则能亨通。

在具体应用上，蒙卦提示我们：在事业开始时可能会遇到混乱和危机，需要通过勇敢坚毅的行动来扭转局面。在经商和求名方面，强调谨慎和纯正的动机。在婚恋和决策上，则提醒我们要注重品德考察和适时行动。
Content: {
  "content": "屯卦",
  "summary": "在周易中，屯卦象征着万物初生的艰难阶段。它由下卦震（雷）和上卦坎（水）组成，形成云雷交加的景象。卦辞'元，亨，利，贞'表明虽然初始吉利，但'勿用有攸往'提示此时不宜妄动，而'利建侯'则指出这是奠定基础的时机。\n\n《象辞》通过云雷之象，启示君子应当效法这种自然现象，既要像云一样广施恩泽，又要如雷一般展现威严，以此治理事务。北宋邵雍将其解读为万物始生必经的困难阶段，预示着先苦后甜的人生历程。\n\n传统解卦强调这是充满挑战的时期：震卦代表行动，坎卦象征险阻，两者结合形成动而遇险之象。正如植物破土而出时的艰难，任何新生事物都会经历这个充满阻力的阶段。但若能顺应时势，终将迎来蓬勃发展。\n\n在具体应用方面：时运建议坚守待时；财运预示创业艰辛；家宅需要修整；健康重在调养。事业上需要知难而进，经商则要坚定信念，婚恋可能好事多磨。决策时既要保持坚韧，又需寻求外援，以乐观态度面对暂时的困境。"
}
``
Summary: {
  "content": "屯卦",
  "summary": "在周易中，屯卦象征着万物初生的艰难阶段。它由下卦震（雷）和上卦坎（水）组成，形成云雷交加的景象。卦辞'元，亨，利，贞'表明虽然初始吉利，但'勿用有攸往'提示此时不宜妄动，而'利建侯'则指出

### 异常分析


训练第一个 epoch 时，Training Loss 比较奇怪：

```
Step	Training Loss
1	3.594100
2	4.049100
3	3.091200
4	3.381700
5	3.547800
6	2.610200
7	2.657900
8	3.163900
```

通过解析 gpt-4o-mini 生成结果发现问题

In [23]:
def gen_data(raw_content):
    """
    使用LangChain GPT-4o-mini调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-4o-mini模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:"{卦名}"
        summary:"{内容}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [24]:
# 执行主函数
main()

Content: 蒙卦
Summary: 在周易中，蒙卦是一个充满启蒙智慧的卦象，由下卦坎（水）和上卦艮（山）组成。这一卦象象征着蒙昧与启蒙的过程，正如山下涌出的泉水，代表着初生的智慧与潜力。蒙卦的核心哲学是：启蒙不是由智者强加于愚者，而是愚者主动寻求智者的指引。第一次真诚的求问会得到回应，但反复轻慢的询问则会被忽视。

蒙卦的卦辞“亨。匪我求童蒙，童蒙求我”强调了启蒙的主动性。它告诉我们，真正的智慧来自于内心的求知欲和对外界指导的尊重。在人生旅途中，我们常常会处于蒙昧状态，这时需要像君子一样，以果敢的行动和坚定的品德来培育自己。

从《断易天机》和邵雍的解释来看，蒙卦虽主蒙昧疑惑，但通过接受贤师良友的教导，可以开启智慧，转凶为吉。傅佩荣则从实际应用角度指出，此卦在时运、财运、家宅和健康方面都有积极意义，关键在于果断行动和品德修养。

传统解卦进一步阐释，蒙卦象征着人生或事业初期的混乱与危机，唯有通过勇敢坚毅的行动和脚踏实地的努力才能扭转局面。在经商、求名、婚恋和决策等方面，蒙卦提醒我们要谨慎行事，注重道德修养，接受教育，等待适当时机，方能一帆风顺。
Content: 屯卦
Summary: 在周易中，屯卦象征着万物初生时的艰难与希望。它由下卦震（雷）和上卦坎（水）组成，形成云雷交加的景象，寓意着创始时期的困难与挑战。卦辞'元，亨，利，贞'表明，虽然初始艰难，但最终会通达顺利。

屯卦的核心哲学是：在事物初创阶段，面对险阻需要谨慎行事。卦象显示'勿用，有攸往，利建侯'，提示此时不宜冒进，但有利于奠定基础。君子观此卦象，应当效法云雷之势，既要有雷霆般的决断力，又要具备云雨般的润泽之功。

从运势来看，屯卦预示着'先劳后逸，苦尽甘来'的历程。无论是事业、财运还是婚恋，初期都会遇到阻碍，需要坚韧不拔的毅力和灵活应变的能力。邵雍解为'万物始生，开始困难'，傅佩荣则强调'宜守不宜进'的处世之道。

值得注意的是，屯卦虽显示艰难，但蕴含着巨大的发展潜力。只要能够坚持正道，终将'排除困难，方可通达'。这提醒我们，在面对人生或事业的初创阶段时，既要认清困难，又要保持信心，最终必能迎来欣欣向荣的局面。
Content: 需卦
Summary: 在周易哲学体系中，需卦（坎上乾下）蕴含着深刻的等待智慧。此卦上卦为坎（水），象征云雨和险阻；下卦为乾（天），代表刚健和光明。卦象呈现'云上于天